
## Installing Libraries
---



In [13]:
# Colab specific
from google.colab import drive
from google.colab import files
import os

drive.mount('/content/drive')

#change the path
os.chdir('/content/drive/MyDrive/MIC/data_picrust')

Mounted at /content/drive


In [5]:
!pip install biopython
from IPython import get_ipython
from IPython.display import display
!pip install ete3

!pip install dendropy

ERROR: Could not find a version that satisfies the requirement python3-pyqt5 (from versions: none)
ERROR: No matching distribution found for python3-pyqt5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.5/459.5 kB 6.4 MB/s eta 0:00:00


In [11]:
!pip install pyqt5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 28.2 MB/s eta 0:00:00


In [3]:
# Standard library imports
import os
import json
import tempfile
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Generator
from dataclasses import dataclass
from math import log
from random import choice
from datetime import datetime
import shutil
import logging
from typing import Tuple, List, Dict
import re

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image

# Biopython imports
from Bio import Entrez, SeqIO, AlignIO, Phylo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Phylo.Consensus import majority_consensus

# QIIME2 imports
#import qiime2
#from qiime2.plugins import demux, dada2, phylogeny, alignment, diversity
#import qiime2.plugins.feature_table.methods as feature_table_methods
#import qiime2.plugins.feature_table.visualizers as visualizers

# biom imports
#from biom.table import Table
#from biom.util import biom_open
#from biom import load_table

# Tree visualization imports
from ete3 import (
    Tree as EteTree,  # Renamed to avoid conflict
    TreeStyle,
    NodeStyle,
    TextFace,
    RectFace,
    faces,
    CircleFace
)
import dendropy
from dendropy import Tree as DendroTree  # Renamed to avoid conflict

# Subprocess for external commands
import subprocess

In [4]:
'''# For VSCode
output_dir = Path("/home/beatriz/MIC/2_Micro/data_qiime")
# Create output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)
bootstrap_dir = output_dir / "bootstrapping"
bootstrap_dir.mkdir(exist_ok=True)

input_dir = Path("/home/beatriz/MIC/2_Micro/data_tree")
original_sequences_file = input_dir / "final_sequences_integrate.fasta"'''
# for colab/kaggle
abundance_excel= Path("/content/drive/MyDrive/MIC/data_picrust/merged_to_sequence.xlsx")
large_dir = Path("/content/drive/MyDrive/MIC/")
input_galaxy = large_dir / "data_galaxies"
data_qiime = Path("/content/drive/MyDrive/MIC/data_qiime")
#output_dir = Path("/home/beatriz/MIC/2_Micro/data_qiime/output_dir")
#output_dir.mkdir(parents=True, exist_ok=True)


In [5]:
source_type = 'check_core_usual' # made on book 4 directory /data_ref

In [6]:
#selected = pd.read_excel("data/finalist_dfs.xlsx", sheet_name='core_taxa', header=[0,1,2,3,4,5,6,7]) # for individual or two combinations activate
selected = pd.read_excel(abundance_excel, sheet_name='core_check_usual_taxa', header=[0,1,2,3,4,5,6,7]) # for the mixture of the 3 df
# Drop first row (index 0) and first column in one chain
selected = selected.drop(index=0).drop(selected.columns[0], axis=1)
# Remove 'Unnamed' level names
selected.columns = selected.columns.map(lambda x: tuple('' if 'Unnamed' in str(level) else level for level in x))
# If the dataframe has Nan in sites it will replace it with Source
selected['Sites'] = selected['Sites'].fillna('Source')
# Fill the other index with nothing
selected = selected.fillna(' ')
# Setting index to Sites
selected= selected.set_index("Sites")
selected_taxa = selected.T

In [7]:
# First, let's create a new DataFrame with the correct structure
selected_taxa= selected_taxa.reset_index()

# Now, rename the columns
current_columns = selected_taxa.columns.tolist()
new_columns = ['Jointax', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'GID']
combined_columns = new_columns + current_columns[8:]
selected_taxa.columns = combined_columns

# Now we can create the dictionaries as before
category_dict = selected_taxa.iloc[0, 8:].to_dict()
jointax_dict = selected_taxa[['Genus', 'Jointax']].set_index('Genus')['Jointax'].to_dict()

Taxa should be on multiindex and sites are headers

In [18]:
selected_taxa.head()

,Jointax,Kingdom,Phylum,Class,Order,Family,Genus,GID,site_1,site_2,...,site_62,site_63,site_64,site_65,site_66,site_67,site_68,site_69,site_70,Source
0,Category,,,,,,,,3.0,1.0,...,2.0,2.0,2.0,2.0,2.0,3.0,3.0,1.0,1.0,
1,Rhodocyclales_Rhodocyclaceae_Azospira,Bacteria,Proteobacteria,Betaproteobacteria,Rhodocyclales,Rhodocyclaceae,Azospira,110,26.928048,1.85923,...,0.571304,0.624133,0.26,4.518236,0.4,0.004886,0,1.47,1.72,chk-core
2,Actinomycetales_Dermabacteraceae_Brachybacterium,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Dermabacteraceae,Brachybacterium,140,0,0,...,0,0,0,0.054437,0,0,0.021172,0,0,chk
3,Actinomycetales_Brevibacteriaceae_Brevibacterium,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Brevibacteriaceae,Brevibacterium,145,0,0,...,0,0,0,0.217746,0,0,0,0,0,chk
4,Erysipelotrichales_Erysipelotrichaceae_Bulleidia,Bacteria,Firmicutes,Erysipelotrichi,Erysipelotrichales,Erysipelotrichaceae,Bulleidia,154,0,0,...,0,0,0,0,0,0,0,0,0,chk


In [8]:
def get_color_scheme(genus_name: str) -> str:
    """
    Color scheme for bacterial categories:
    - checked (chk): #000080 (navy blue)
    - usual (us): #8B0000 (dark red)
    - chk-us: #9370DB (purple)
    - chk-core: #000033 (darker navy)
    - core-us: #654321 (dark brown)
    - chk-core-us: #4A0404 (deep burgundy)
    """
    if 'Source' not in selected_taxa.columns:
        return 'black'

    try:
        genus_source = selected_taxa.loc[selected_taxa['Genus'] == genus_name, 'Source'].iloc[0]
        source = str(genus_source).lower()
        #chk-core', 'chk', 'chk-core-us', 'chk-us', 'core-us

        if 'chk-core-us' in source:
            return '#4A0404'  # Deep burgundy for all three
        elif 'chk-core' in source:
            return '#000033'  # Darker navy for checked+core
        elif 'core-us' in source:
            return '#654321'  # Dark brown for core+usual
        elif 'chk-us' in source:
            return '#9370DB'  # Purple for checked+usual
        elif 'chk' in source:
            return '#000080'  # Navy blue for checked only
        elif 'us' in source:
            return '#8B0000'  # Dark red for usual only
        return 'black'

    except (IndexError, KeyError):
        return 'black'
# Define set to track added names
added_names = set()

In [9]:
def build_aggregated_data(selected_taxa, category_dict):
    """
    Aggregate abundance data by category for each genus

    Args:
        selected_taxa (pd.DataFrame): DataFrame with taxa and abundance data
        category_dict (dict): Mapping of sites to categories

    Returns:
        dict: Dictionary mapping genera to their abundance in each category
    """
    # Initialize empty dictionary to store the aggregated data
    aggregated_data = {}

    # Loop through each row in the DataFrame
    for index, row in selected_taxa.iterrows():
        genus = row['Genus'] # Using level_6 for genus
        if genus not in aggregated_data:
            aggregated_data[genus] = {'category_1': 0, 'category_2': 0, 'category_3': 0}

        # Loop through each site column
        site_columns = [col for col in selected_taxa.columns if col.startswith('site_')]
        for site_column in site_columns:
            category = category_dict[site_column]
            abundance = row[site_column]

            if category == 1:
                aggregated_data[genus]['category_1'] += abundance
            elif category == 2:
                aggregated_data[genus]['category_2'] += abundance
            elif category == 3:
                aggregated_data[genus]['category_3'] += abundance

    return aggregated_data

In [10]:
aggregated_data = build_aggregated_data(selected_taxa, category_dict)

In [11]:
def create_tree_visualization(tree_path, aggregated_data, category_colors, categories):
    """
    Creates and styles a phylogenetic tree visualization

    Args:
        tree_path (str): Path to the newick tree file
        aggregated_data (dict): Dictionary of abundance data by genus and category
        category_colors (dict): Color mapping for categories
        categories (dict): Category descriptions

    Returns:
        tuple: (EteTree, TreeStyle) - styled tree and its styling
    """
    # Load the tree
    t = EteTree(tree_path, format=0)

    # Create tree style
    ts = TreeStyle()
    ts.show_leaf_name = False
    ts.branch_vertical_margin = 5
    ts.scale = 120
    ts.show_branch_support = False

    # Create node style
    nstyle = NodeStyle()
    nstyle["size"] = 5
    nstyle["fgcolor"] = "#666666"
    nstyle["hz_line_width"] = 2
    nstyle["vt_line_width"] = 2

    # Bootstrap label layout function
    def layout(node):
        if not node.is_leaf():
            if hasattr(node, "support"):
                support_value = node.support * 100 if node.support <= 1 else node.support
                bootstrap_face = TextFace(f"{support_value:.0f}%", fsize=8, fgcolor="braun")
                node.add_face(bootstrap_face, column=0, position="branch-top")

    ts.layout_fn = layout

    # Track added names
    added_names = set()

    # Apply style and add annotations
    for node in t.traverse():
        if node.is_leaf() and node.name in aggregated_data:
            name_parts = node.name.split('(')
            clean_name = name_parts[0].strip()
            node_color = get_color_scheme(clean_name)

            if clean_name not in added_names:
                name_face = TextFace(clean_name, fsize=10, fgcolor=node_color)
                node.add_face(name_face, column=1, position="branch-right")
                added_names.add(clean_name)

                node.img_style["fgcolor"] = node_color

                if len(name_parts) > 1:
                    bootstrap = name_parts[1].rstrip(')')
                    bootstrap_face = TextFace(f"({bootstrap})", fsize=8)
                    node.add_face(bootstrap_face, column=1, position="branch-top")

                column = 2
                for category, abundance in aggregated_data[clean_name].items():
                    if abundance > 0:
                        circle = CircleFace(radius=3 + (abundance/5),
                                         color=category_colors[category],
                                         style="sphere")
                        node.add_face(circle, column=column, position="branch-right")
                        column += 1

    # Add legend
    legend_faces = []
    for cat_num, desc in categories.items():
        legend_face = CircleFace(5, category_colors[f'category_{cat_num[-1]}'])
        legend_faces.append(TextFace(f"  {cat_num} - {desc}", fsize=8))
        legend_faces.append(legend_face)
        legend_faces.append(TextFace("   "))

    ts.legend_position = 3
    ts.legend.add_face(TextFace("System Categories", fsize=10, bold=True), column=0)
    for idx, face in enumerate(legend_faces):
        ts.legend.add_face(face, column=idx % 3)

    return t, ts

In [12]:
print(tree_path)

NameError: name 'tree_path' is not defined

In [ ]:
# Rendering the tree visualisation
if __name__ == "__main__":

    # Define colors and categories
    category_colors = {
        'category_1': '#008800',
        'category_2': '#FF8C00',
        'category_3': '#FF0000'
    }

    categories = {
        'category_1': 'Normal Operation',
        'category_2': 'Early Warning',
        'category_3': 'System Failure'
    }

    # Create and render tree
    tree_path = data_qiime / "consensus_tree_integrated_ncbi.newick"
    t, ts = create_tree_visualization(str(tree_path), aggregated_data, category_colors, categories)

    # Render final visualization
    t.render("data_qiime/iTOL_visualisation_MAFF_NCBI_new.png", w=1000, h=3050, dpi=300, tree_style=ts)

## Galaxy Tree
### Extracting the Genera of core_usual_checked taxa dataframe

In [ ]:
input_galaxy = Path("/home/beatriz/MIC/data_galaxies")
newick_path =  input_galaxy / "Galaxy7-PICRUSt2-Full-pipeline-on-data-2-and-data-1-Tree-reference-study-16S-sequences.newick"
# Load tree
tree_galaxy = Phylo.read(newick_path, "newick")

output_base = Path("/home/beatriz/MIC/2_Micro/data_picrust/output_base")
otu_path = output_base / "otu_mapping.tsv"

otu_mapping = pd.read_csv(otu_path, sep="\t")
taxon_ids = otu_mapping["taxon"].tolist()

# Get terminals that match your IDs
terminals_to_keep = []
for terminal in tree_galaxy.get_terminals():
    if terminal.name in taxon_ids:
        terminals_to_keep.append(terminal)

# Collect only the filtered terminal names (as strings)
filtered_names = {terminal.name for terminal in terminals_to_keep}

# Prune the tree: remove terminals not in filtered_names
for terminal in list(tree_galaxy.get_terminals()):
    if terminal.name not in filtered_names:
        node_to_prune = tree_galaxy.find_any(name=terminal.name)
        if node_to_prune:
            tree_galaxy.prune(node_to_prune)

data_qiime = Path("/home/beatriz/MIC/2_Micro/data_qiime")
# Write the pruned tree
filter_path = data_qiime / "filter_galaxy.newick"
Phylo.write(tree_galaxy, filter_path, "newick")

ValueError: can't find a matching target below this root

### Replacing the Taxon Names on the Galaxy Newick Tree

In [ ]:
# Read the pruned tree
with open(filter_path, 'r') as f:
    subtree_galaxy = f.read()

# Convert the mapping DataFrame to a dictionary for easier lookup
taxon_to_genus = dict(zip(otu_mapping['taxon'], otu_mapping['Genus']))
str_taxon_to_genus = {str(k): v for k, v in taxon_to_genus.items()}


# Define a function to replace taxon IDs with genus names
def replace_taxon_with_genus(match):
    taxon_id_str = (match.group(1))
    if taxon_id_str in str_taxon_to_genus:
        return str_taxon_to_genus[taxon_id_str] + ":"
    # Try integer matching as backup
    try:
        taxon_id_int = int(taxon_id_str)
        if taxon_id_int in taxon_to_genus:
            return taxon_to_genus[taxon_id_int] + ":"
    except (ValueError, TypeError):
        pass

    return match.group(0)

# This pattern looks for numbers followed by a colon
pattern = r"[']?(\d+)[']?(?=:)"
# Replace IDs with genus names
subtreeG_galaxy = re.sub(pattern, replace_taxon_with_genus, subtree_galaxy)


# Write final tree
gen_tree_path = data_qiime / "galaxy_tree.newick"
# Write tree with genus
with open(gen_tree_path, 'w') as f:
    f.write(subtreeG_galaxy)

In [ ]:
print(subtreeG_galaxy)

((((((((643348582:0.00000,2263196004:0.00369):0.14188,2574180439:0.11545):0.15517,(646311953:0.05300,2519899621:0.16935):0.12590):0.32638,646311962:0.26789):0.19678,(((2739367826:0.64128,(((2514752028:0.24381,(2513020048:0.00000,2540341085:0.00000):0.09948):0.28933,2615840657:0.25542):0.10365,((((2706794980-cluster:0.18347,(((2264265199:0.40158,(2706794984-cluster:0.00182,2708742482:0.00000):0.13184):0.03729,(2706794976:0.13381,2706794986:0.18904):0.02834):0.01661,2703719290-cluster:0.17587):0.04313):0.04476,(((((646564526:0.00000,2540341062:0.00000):0.00000,2597490266-cluster:0.00000):0.00000,((640427111:0.00000,2540341061:0.00000):0.00000,2675903542:0.00000):0.00181):0.01143,((637000089:0.00000,2636415841-cluster:0.00000):0.00933,(2651869856:0.00183,(2639762808-cluster:0.00000,(2554235449:0.00185,646311919:0.00000):0.00000):0.00000):0.00541):0.01521):0.17254,(648028022:0.07839,2619618830:0.05721):0.12756):0.15166):0.08717,(2728369433:0.20417,2602041625:0.41181):0.04640):0.09089,(2703

In [ ]:
# Rendering the tree visualisation
if __name__ == "__main__":

    # Define colors and categories
    category_colors = {
        'category_1': '#008800',
        'category_2': '#FF8C00',
        'category_3': '#FF0000'
    }

    categories = {
        'category_1': 'Normal Operation',
        'category_2': 'Early Warning',
        'category_3': 'System Failure'
    }

    # Create and render tree

t, ts = create_tree_visualization(subtreeG_galaxy, aggregated_data, category_colors, categories)

# Render final visualization
t.render("data_qiime/iTOL_visualisation_galaxy.png", w=1000, h=3050, dpi=300, tree_style=ts)

{'nodes': [[2.323435555061033,
   2235.1211509035656,
   2.609396854145468,
   2235.1693124403064,
   0,
   None],
  [6.072216609278524,
   1421.8617334800329,
   6.358177908362959,
   1421.9098950167734,
   1,
   None],
  [8.574377976267328,
   6.244667653583394,
   8.860339275351764,
   6.292829190323948,
   2,
   None],
  [11.076539343256133,
   3.4289462078500605,
   11.362500642340569,
   3.4771077445906147,
   3,
   None],
  [13.578700710244938,
   0.7043624748306028,
   13.864662009329374,
   0.7525240115711568,
   4,
   None],
  [16.5931331484136,
   0.48161536740554034,
   16.879094447498037,
   0.5297769041460944,
   5,
   None],
  [19.095294515402404,
   0.2829490283507549,
   19.381255814486842,
   0.331110565091309,
   6,
   None],
  [21.59745588239121,
   0.18662595486964687,
   21.883417181475647,
   0.2347874916102009,
   7,
   None],
  [21.811926856704538,
   0.13846441812909285,
   22.097888155788972,
   0.18662595486964687,
   8,
   None],
  [21.843582772513184,
   0